In [5]:
from system import *
s1 = system('O-x-O',MMA=True,ManyVariables=False,TwoPhotonResonance= True)

Initializing system O-x-O  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblad operators ...

System  O-x-O  initialized in 28.8 seconds.


# Effective Hamiltonian as Real part

Will output $Q_n$ from expression:

$$\Delta_n = - \dfrac{\Omega^2}{\gamma} \text{Re}\{Q_n \} $$

In [8]:
diag = 1  #input n corresponding to diag element

var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g','gamma0','De0','g0','R0',domain='positive')
var('DEt',domain='positive',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='positive',  latex_name =r'\tilde{\Delta }_e')
var('De0t',domain='positive',  latex_name =r'\tilde{\Delta }_{e0}')
var('dct',domain='positive',  latex_name =r'\tilde{\delta }_c')
var('DEg',domain='positive',  latex_name =r'\Delta_{E\gamma}')
var('Deg',domain='positive',  latex_name =r'\Delta_{e\gamma}')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c',domain='positive',  latex_name =r'c')

#(gamma_f=gamma)#gamma_g=0,

#Take only one of the compl conj terms. Switch maybe second 0 to 1
min_expr_len = 10**10
for term in range(2):
    b = s1.eff_hamiltonian_gs[diag,diag]

    
    b = b.operands()[1].operands()[term]  #  + b.operands()[1].operands()[3] use if obtained with sage


    b = b.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    b = b.subs(De=+0.5*I*gamma   +Det*gamma)
    
    b = b.subs(De0= +0.5*I*gamma0 +De0t*R0*gamma)
    
    #var('a0', latex_name =r'a_0')
    #b = b.subs(De0=+0.5*I*gamma   +Det*gamma*a0)
    #var('n')
    b = b.subs(DE=DEg*gamma)
    b = b.subs(De=Deg*gamma)
    var('a0', latex_name =r'a_0')
    b = b.subs(De0=a0*Deg*gamma)
    
    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))
    
    var('R0',domain='positive',  latex_name =r'R_0')
    b = b.subs(g0=g*sqrt(R0))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)


    var('r_g',domain='real',latex_name =r'r_g')
    b = b.subs(gamma_g=gamma*r_g)


    var('r_f',domain='real',latex_name =r'r_f')
    b = b.subs(gamma_f=gamma*r_f)
    
    var('r0',domain='real',latex_name =r'r_0')
    b = b.subs(gamma0=gamma*r0)


    b = b.subs(g = sqrt(C*kappa_c*gamma)) 
    
    b = b.subs(dc = (dct+0.5*I)*kappa_c )
    
    b = b.subs(r_b = c/(C*R_v))
    
    b = b*gamma  #Omega has to be substituted with Omega/gamma
    

    b = b._mathematica_().Factor()._sage_()
    b = MMA_simplify(b,full= False)
    #holds the smallest expression
    if len(str(b))<min_expr_len:
        min_expr_len = len(str(b))
        h_eff = b

show( (symround(h_eff,digits=1)))

symround: Deleted coefficient 4.9e-16


((4*c^2 + 3*c + 0.25)*C*(De0t + Det) - (-6*I*c^2 - 2*I*c - 0.12*I)*De0t*Det - C^2*(4*I*c + 0.50*I))/((4*De0t*Det*(c + 0.25)^2 - (2*I*De0t*c + 2*I*Det*c + 0.50*I*De0t + 0.50*I*Det)*C - C^2)*C*R_f + ((4*c^2 + 3*c + 0.25)*C*(De0t + Det) - (-6*I*c^2 - 2*I*c - 0.12*I)*De0t*Det - C^2*(4*I*c + 0.50*I))*DEt)

In [59]:
#show(symround(MMA_simplify(h_eff.subs(De0t=I/2,R0=0),full= not True).denominator()*48))

In [11]:
factor = -8 #*16*4


print('numerator')
nu = symround(SR(str(MMA_simplify(factor*(h_eff.numerator().expand()),full=not True)\
                      ._mathematica_().Collect('DEt')._sage_()).replace('Sqrt','sqrt'))\
              ,digits=3)
nu = nu.collect(R0)
show(nu)
print("\n\n Latex")
print(latex(nu))

print("\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print('\n\ndenominator')
den = symround(SR(str(MMA_simplify(factor*(h_eff.denominator().expand()),full= not True)\
                      ._mathematica_().Collect('DEt')._sage_()).replace('Sqrt','sqrt'))\
               ,digits=3)

show(den)
print("\n\n Latex")
print(latex(den))

numerator


2*(16*c^2 + 12*c + 1)*C*(De0t + Det) + (48*I*c^2 + 16*I*c + I)*De0t*Det + C^2*(-32*I*c - 4*I)



 Latex
2 \, {\left(16 \, {c}^{2} + 12 \, {c} + 1\right)} C {\left({\tilde{\Delta }_{e0}} + {\tilde{\Delta }_e}\right)} + {\left(48 i \, {c}^{2} + 16 i \, {c} + i\right)} {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} + C^{2} {\left(-32 i \, {c} - 4 i\right)}


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


denominator


TypeError: Error executing code in Mathematica
CODE:
	sage3=;
Mathematica ERROR:
	Syntax::sntxf: "sage3=" cannot be followed by ";".

# Expand linearly in the limit:   $C\rightarrow \infty$

In [61]:
#with mathematica
q = b.subs(C=1/x)
q = MMA_simplify(q)
tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
tayl = tayl.subs(x=1/C)
#t = q._mathematica_().Series((x,0,2))
#t_exp = t[3]
#t_st = t[4]
#tayl = 0
#for i in range(1,len(t_exp)):
#    tayl += MMA_simplify( t_exp[i]._sage_() ) / C**(i-1+Integer(t[4]))

show(symround(tayl))

#with sage
#tayl = b.taylor(C_c,infinity, 1)
#show(symround(MMA_simplify(tayl)))

((2*I*R_v*c_k + 0.66*I)*R_v*c_k + 0.042*I)/(((2*I*DE + 1.3*R_f)*R_v^2*c_k^2 + (0.66*I*DE + R_f)*R_v*c_k + 0.042*I*DE + 0.083*R_f)*C)

# Lindblad Operators

Choose which of the Lindblau operators to view and which non zero element of it

In [7]:
def lind(lind_op,which):    
    L_matrix_orig = s1.lindblau_list[lind_op]
    L_matrix      = s1.eff_lindblau_list[lind_op] #lindblau_list[lind_op] ##(gamma_g=0,gamma_f=gamma)

    L_nonzeros = []
    L_nonzeros_pos = []
    #find non zero elements
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not L_matrix[i,j].is_zero():
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))

    #show what these elements do 
    #for t in L_nonzeros_pos:
    #    print(f' |{s1.gs_e1_dec_states[t[0]]}> < {s1.gs_e1_dec_states[t[1]]} | ')

    L_elem = L_nonzeros[which]

    var('gamma_f')
    L_elem = L_elem.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    L_elem = L_elem.subs(De=+0.5*I*gamma   +Det*gamma)
    
    #L_elem = L_elem.subs(DE=gamma*DEg)
    #L_elem = L_elem.subs(De=gamma*Deg)                     
    #COUPLINGS
    b = L_elem
    
    b = b.subs(De0= +0.5*I*gamma0 +De0t*gamma)
    
    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))
    
    var('R0',domain='positive',  latex_name =r'R_0')
    b = b.subs(g0=g*sqrt(R0))
    
    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)


    var('r_g',domain='real',latex_name =r'r_g')
    b = b.subs(gamma_g=gamma*r_g)


    var('r_f',domain='real',latex_name =r'r_f')
    b = b.subs(gamma_f=gamma*r_f)


    b = b.subs(g = sqrt(C*kappa_c*gamma)) 
    
    b = b.subs(r_b = c/(C*R_v))
    
    b = b.subs(dc = (dct+0.5*I)*kappa_c )
    
    
    L_elem = b
    
  
    L_elem = L_elem._mathematica_().PowerExpand()._sage_()
    L_elem = L_elem._mathematica_().Factor()._sage_()


    L_elem = MMA_simplify(L_elem,full=True)

    L_elem = SR(str(L_elem).replace('Sqrt','sqrt'))

    

    show( symround(L_elem))

    print(f' |{s1.gs_e1_dec_states[L_nonzeros_pos[which][0]]}> < {s1.gs_e1_dec_states[L_nonzeros_pos[which][1]]} | \n')
    print(f' Original coefficient :')
    show(s1.L_coeffs[lind_op])
    return L_elem


# View

In [37]:
lind_op = 2  #which lindblau operator
which   = 2  #which non zero element to access
L_elem = lind(lind_op,which)

print(latex(symround(L_elem)))


IndexError: list index out of range

In [30]:
lind_op = 0  #which lindblau operator
which   = 0  #which non zero element to access
L_elem = lind(lind_op,which)

print(latex(symround(L_elem)))


symround: Deleted coefficient 2.2e-16
symround: Deleted coefficient 1.8e-17


(0.50*I*C*R0 - De0t*(c + 0.25))*sqrt(C)*De0t*Omega*sqrt(R_f)*c*e^(-I*phi)/(((-0.25*C^2*R0^2 + De0t^2*(c + 0.25)^2 + C*De0t*R0*(-I*c - 0.25*I))*C*R_f - (C^2*R0^2*(I*c + 0.12*I) - ((2*c + 1.5)*c + 0.12)*C*De0t*R0 - De0t^2*(c + 0.25)*(1.5*I*c + 0.12*I))*DEt)*sqrt(gamma))

 |0000f000> < 0000g000 | 

 Original coefficient :


sqrt(kappa_c)

symround: Deleted coefficient 2.2e-16
symround: Deleted coefficient 1.8e-17
\frac{{\left(0.50i \, C {R_0} - {\tilde{\Delta }_{e0}} {\left({c} + 0.25\right)}\right)} \sqrt{C} {\tilde{\Delta }_{e0}} {\Omega} \sqrt{R_{f}} {c} e^{\left(-i \, {\phi}\right)}}{{\left({\left(-0.25 \, C^{2} {R_0}^{2} + {\tilde{\Delta }_{e0}}^{2} {\left({c} + 0.25\right)}^{2} + C {\tilde{\Delta }_{e0}} {R_0} {\left(-i \, {c} - 0.25i\right)}\right)} C R_{f} - {\left(C^{2} {R_0}^{2} {\left(i \, {c} + 0.12i\right)} - {\left({\left(2 \, {c} + 1.5\right)} {c} + 0.12\right)} C {\tilde{\Delta }_{e0}} {R_0} - {\tilde{\Delta }_{e0}}^{2} {\left({c} + 0.25\right)} {\left(1.5i \, {c} + 0.12i\right)}\right)} {\tilde{\Delta }_E}\right)} \sqrt{{\gamma}}}


In [31]:
factor = 1 #*16*4


print('numerator')
nu = symround(SR(str(MMA_simplify(factor*(L_elem.numerator().expand()),full=not True)\
                      ._mathematica_().Collect('R_0')._sage_()).replace('Sqrt','sqrt'))\
              ,digits=3)#collect(R0??)
nu = nu.collect(R0)
show(nu)
print("\n\n Latex")
print(latex(nu))

print("\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print('\n\ndenominator')
den = symround(SR(str(MMA_simplify(factor*(L_elem.denominator().expand()),full= not True)\
                      ._mathematica_().Collect('R0')._sage_()).replace('Sqrt','sqrt'))\
               ,digits=3)   #collect(R0??)

show(den)
print("\n\n Latex")
print(latex(den))

numerator


0.500*I*C^1.50*De0t*Omega*R0*sqrt(R_f)*c*e^(-I*phi) - sqrt(C)*De0t^2*Omega*sqrt(R_f)*c^2*e^(-I*phi) - 0.250*sqrt(C)*De0t^2*Omega*sqrt(R_f)*c*e^(-I*phi)



 Latex
0.500i \, C^{1.50} {\tilde{\Delta }_{e0}} {\Omega} {R_0} \sqrt{R_{f}} {c} e^{\left(-i \, {\phi}\right)} - \sqrt{C} {\tilde{\Delta }_{e0}}^{2} {\Omega} \sqrt{R_{f}} {c}^{2} e^{\left(-i \, {\phi}\right)} - 0.250 \, \sqrt{C} {\tilde{\Delta }_{e0}}^{2} {\Omega} \sqrt{R_{f}} {c} e^{\left(-i \, {\phi}\right)}


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


denominator
symround: Deleted coefficient 2.22e-16
symround: Deleted coefficient 7.40e-17
symround: Deleted coefficient 4.63e-18


(-0.250*C^3*R_f + C^2*DEt*(-I*c - 0.125*I))*R0^2*sqrt(gamma) + (C^2*De0t*R_f*(-I*c - 0.250*I) + (2*c^2 + 1.50*c + 0.125)*C*DEt*De0t)*R0*sqrt(gamma) + (C*De0t^2*R_f*(c + 0.250)^2 + (1.50*I*c^2 + 0.500*I*c + 0.0312*I)*DEt*De0t^2)*sqrt(gamma)



 Latex
{\left(-0.250 \, C^{3} R_{f} + C^{2} {\tilde{\Delta }_E} {\left(-i \, {c} - 0.125i\right)}\right)} {R_0}^{2} \sqrt{{\gamma}} + {\left(C^{2} {\tilde{\Delta }_{e0}} R_{f} {\left(-i \, {c} - 0.250i\right)} + {\left(2 \, {c}^{2} + 1.50 \, {c} + 0.125\right)} C {\tilde{\Delta }_E} {\tilde{\Delta }_{e0}}\right)} {R_0} \sqrt{{\gamma}} + {\left(C {\tilde{\Delta }_{e0}}^{2} R_{f} {\left({c} + 0.250\right)}^{2} + {\left(1.50i \, {c}^{2} + 0.500i \, {c} + 0.0312i\right)} {\tilde{\Delta }_E} {\tilde{\Delta }_{e0}}^{2}\right)} \sqrt{{\gamma}}


In [35]:
show(symround(L_elem.subs(R0=0)))

symround: Deleted coefficient 2.2e-16
symround: Deleted coefficient 1.8e-17


-sqrt(C)*De0t^2*Omega*sqrt(R_f)*(c + 0.25)*c*e^(-I*phi)/((C*De0t^2*R_f*(c + 0.25)^2 + DEt*De0t^2*(c + 0.25)*(1.5*I*c + 0.12*I))*sqrt(gamma))

In [29]:
latex(symround(SR(str(MMA_simplify(factor*(L_elem.numerator().expand()),full=not True)).replace('Sqrt','sqrt')),digits=3) / (symround(SR(str(MMA_simplify(factor*(L_elem.denominator().expand()),full=not True)).replace('Sqrt','sqrt')),digits=3)))

\frac{32 \, \sqrt{C} {\tilde{\Delta }_e}^{2} {\Omega} \sqrt{R_{f}} {c}^{2} e^{\left(2 i \, {\phi}\right)}}{{\left({\left(2 \, {\left(16 \, {c}^{2} + 12 \, {c} + 1\right)} {\tilde{\Delta }_e}^{2} - C {\tilde{\Delta }_e} {\left(48 i \, {c} + 8 i\right)} - 8 \, C^{2}\right)} C R_{f} + {\left(4 \, {\left(16 \, {c}^{2} + 10 \, {c} + 1\right)} C {\tilde{\Delta }_e} - {\left(-48 i \, {c}^{2} - 16 i \, {c} - i\right)} {\tilde{\Delta }_e}^{2} - C^{2} {\left(16 i \, {c} + 4 i\right)}\right)} {\tilde{\Delta }_E}\right)} \sqrt{{\gamma}}}

In [81]:
s1.e1_dim

31

In [84]:
s1.e1_states

array(['0000f00e', '0000f00X', '0000f010', '0000f011', '0000f100',
       '0000f101', '0000E000', '0000E001', '0001f000', '0001f001',
       '0010f000', '0010f001', '0100f00e', '0100f00X', '0100f010',
       '0100f011', '0100f100', '0100f101', '0100E000', '0100E001',
       '0101f000', '0101f001', '0110f000', '0110f001', '0e00f000',
       '0e00f001', '0X00f000', '0X00f001', '1000f000', '1000f001',
       '1100f001'], dtype='<U32')

In [92]:
for state in s1.e1_states:
    if state[0]=="1":
        print(state)

1000f000
1000f001
1100f001


In [27]:
 s1.states

array(['0g000', '0g001', '0g00e', '0g00o', '0g002', '0g010', '0g011',
       '0g01e', '0g01o', '0g012', '0g100', '0g101', '0g10e', '0g10o',
       '0g102', '0g110', '0g111', '0g11e', '0g11o', '0g112', '0f000',
       '0f001', '0f00e', '0f00o', '0f002', '0f010', '0f011', '0f01e',
       '0f01o', '0f012', '0f100', '0f101', '0f10e', '0f10o', '0f102',
       '0f110', '0f111', '0f11e', '0f11o', '0f112', '0E000', '0E001',
       '0E00e', '0E00o', '0E002', '0E010', '0E011', '0E01e', '0E01o',
       '0E012', '0E100', '0E101', '0E10e', '0E10o', '0E102', '0E110',
       '0E111', '0E11e', '0E11o', '0E112', '1g000', '1g001', '1g00e',
       '1g00o', '1g002', '1g010', '1g011', '1g01e', '1g01o', '1g012',
       '1g100', '1g101', '1g10e', '1g10o', '1g102', '1g110', '1g111',
       '1g11e', '1g11o', '1g112', '1f000', '1f001', '1f00e', '1f00o',
       '1f002', '1f010', '1f011', '1f01e', '1f01o', '1f012', '1f100',
       '1f101', '1f10e', '1f10o', '1f102', '1f110', '1f111', '1f11e',
       '1f11o', '1f1

In [32]:
import collections
l = [item for item, count in collections.Counter(s1.states).items() if count > 1]

In [33]:
seen = set()
uniq = []
for (i,x) in enumerate(s1.states):
    if x in l:
        print(i,x)
    

229 0011E12
239 0011E12
449 0111f12
459 0111f12
889 0o10E12
899 0o10E12
1469 1100f12
1479 1100f12
1769 1e01f12
1779 1e01f12


In [34]:
l

['0011E12', '0111f12', '0o10E12', '1100f12', '1e01f12']